In [1]:
#Download file from MTA
#Input into a PD Dataframe (e.g. correctly label columns)
#Helper tables: Define MTA station Mapping
#reformat PD into a usable DF - Unit ID, Date-time, station name, entries, exits
#sort on unit-id (will sort by time too). remove duplicates. [possibly find a better method]

#- Which station has the most number of units? ... Penn Station (unique turnstile units for the Week of Feb 2nd)
#- What is the total number of entries & exits across the subway system for February 1, 2013? ...neet to calculate net entries and exits [2hrs]
#- Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date? ... add busyness column. filter for only feb 1. group by station [30 mins]
#- What stations have seen the most usage growth/decline in 2013? [NEEDS clarification - since daily variance is significant] ...need to download whole dataset for 2012 and 2013. (speed up calculation)...sum up net transactions for the year [3hrs]
#- What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely? ... [2hrs]
#- Bonus:  What hour is the busiest for station CANAL ST in Q1 2013?



In [124]:
import pandas as pd
import numpy as np
from datetime import datetime

#add station name & count number of turnstiles per station
def addStationName(filename, raw_MTA_data_tmp):
    if 'Station' not in raw_MTA_data_tmp.columns:
        remote_station_mapping = pd.read_excel(filename)
        remote_station_mapping = remote_station_mapping.drop_duplicates(subset=["Remote"]) #assumption only keep the first occuring station name (e.g. 59th st vs. Lexington ave)
        remote_station_mapping["UNIT"] = remote_station_mapping["Remote"]

        raw_MTA_data_tmp = raw_MTA_data_tmp.merge(remote_station_mapping[["UNIT", "Station"]], on=["UNIT"])
    return raw_MTA_data_tmp

def addTimeStamp(vec):
    x = str(vec[0])
    y = str(vec[1])
    try:
        return datetime.strptime(x + " " + y, "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S")
    except:
        return "NULL"

def read_MTA_csv(filename):
    raw_MTA_data = pd.read_csv(filename,error_bad_lines=False)
    raw_MTA_data.columns= ["C/A","UNIT","SCP","DATE1","TIME1","DESC1","ENTRIES1","EXITS1","DATE2","TIME2","DESC2","ENTRIES2","EXITS2","DATE3",
    "TIME3","DESC3","ENTRIES3","EXITS3","DATE4","TIME4","DESC4","ENTRIES4","EXITS4","DATE5","TIME5","DESC5","ENTRIES5","EXITS5","DATE6",
    "TIME6","DESC6","ENTRIES6","EXITS6","DATE7","TIME7","DESC7","ENTRIES7","EXITS7","DATE8","TIME8","DESC8","ENTRIES8","EXITS8"]
    #print(raw_MTA_data.head())

    #Loop through raw dataset. for each row, make an entry in the MTA turnstile data set.
    raw_MTA_data_tmp = raw_MTA_data.copy(deep=True)
    raw_MTA_data_tmp["Unit_ID"] = raw_MTA_data_tmp["C/A"] + raw_MTA_data_tmp["UNIT"] + raw_MTA_data_tmp["SCP"]
    raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["C/A", "SCP"],axis=1)

    for i in range(1,8+1):
        raw_MTA_data_tmp["Datetime_" + str(i)] = raw_MTA_data_tmp[["DATE" + str(i), "TIME" + str(i)]].apply(addTimeStamp,axis=1)

    #raw_MTA_data_tmp["Date_time_1"] = raw_MTA_data_tmp.apply(lambda x: datetime.strptime(x["DATE1"] + " " + x["TIME1"], "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S"),axis=1) 
    raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["DATE1", "TIME1", "DATE2", "TIME2", "DATE3", "TIME3", "DATE4", "TIME4", "DATE5", "TIME5", "DATE6", "TIME6", "DATE7", "TIME7", "DATE8", "TIME8"],axis=1)
    #print(raw_MTA_data_tmp.head())
    return raw_MTA_data_tmp

raw_MTA_data_tmp = read_MTA_csv("~/Projects/NYC_MTA_DATA_ANALYSIS/data/turnstile_130202.txt")
raw_MTA_data_tmp = addStationName("../data/Remote-Booth-Station.xls", raw_MTA_data_tmp)


In [3]:
print("Question 1: 'Which station has the most number of units?'")

MTA_turnstiles_unique = raw_MTA_data_tmp.drop_duplicates(subset=["Unit_ID"]).copy(deep=True)
stationWithMostTerminals = MTA_turnstiles_unique.groupby(by="Station").agg(["count"])["UNIT"].sort_values(by="count",).iloc[-1,:]

print(stationWithMostTerminals)











Question 1: 'Which station has the most number of units?'
count    100
Name: 34 ST-PENN STA, dtype: int64


In [4]:
#convert dataframe so that each row has only 1 4-hour block of data

def expandRows(raw_MTA_data_tmp):
    raw_MTA_data_tmp_x8 = pd.DataFrame()
    for i in range(1,8+1):
        raw_MTA_data_tmp_copy = raw_MTA_data_tmp.copy(deep=True)
        raw_MTA_data_tmp_copy["Unit"] = raw_MTA_data_tmp_copy["Unit_ID"]
        raw_MTA_data_tmp_copy["Datetime"] = raw_MTA_data_tmp_copy["Datetime_" + str(i)]
        raw_MTA_data_tmp_copy["Station"] = raw_MTA_data_tmp_copy["Station"]
        raw_MTA_data_tmp_copy["Cumulative entries"] = raw_MTA_data_tmp_copy["ENTRIES" + str(i)]
        raw_MTA_data_tmp_copy["Cumulative exits"] = raw_MTA_data_tmp_copy["EXITS" + str(i)]
        raw_MTA_data_tmp_copy["Desc"] = raw_MTA_data_tmp_copy["DESC" + str(i)]
        raw_MTA_data_tmp_x8 = pd.concat([raw_MTA_data_tmp_x8,raw_MTA_data_tmp_copy])

    return raw_MTA_data_tmp_x8[raw_MTA_data_tmp_x8.columns[-6:]].copy(deep=True)

raw_MTA_data_tmp = expandRows(raw_MTA_data_tmp)

print(raw_MTA_data_tmp)

             Station               Unit           Datetime  \
0              59 ST   A002R05102-00-00  13-01-27 11:00:00   
1              59 ST   A002R05102-00-00  13-01-28 19:00:00   
2              59 ST   A002R05102-00-00  13-01-30 03:00:00   
3              59 ST   A002R05102-00-00  13-01-31 11:00:00   
4              59 ST   A002R05102-00-00  13-02-01 15:00:00   
...              ...                ...                ...   
30856  RIT-ROOSEVELT  TRAM2R46900-05-01  13-01-29 20:00:00   
30857  RIT-ROOSEVELT  TRAM2R46900-05-01  13-01-30 07:48:11   
30858  RIT-ROOSEVELT  TRAM2R46900-05-01  13-01-31 08:00:00   
30859  RIT-ROOSEVELT  TRAM2R46900-05-01  13-02-01 16:00:00   
30860  RIT-ROOSEVELT  TRAM2R46900-05-01               NULL   

       Cumulative entries  Cumulative exits     Desc  
0               3967826.0         1367578.0  REGULAR  
1               3969685.0         1368236.0  REGULAR  
2               3971424.0         1368820.0  REGULAR  
3               3973086.0         1

In [65]:
#calculate net entries and exits for each turnstile-date and each turnstile

def getNetEntriesExits(MTA_turnstiles):
    #sort & make ID the index...
    MTA_turnstiles["ID"] = MTA_turnstiles["Unit"] + " " + MTA_turnstiles["Datetime"] 
    MTA_turnstiles_sorted = MTA_turnstiles.sort_values(by=["ID"])
    #print(len(MTA_turnstiles_sorted))
    MTA_turnstiles_sorted = MTA_turnstiles_sorted.drop_duplicates(subset=["ID"])
    #print(len(MTA_turnstiles_sorted))

    #calculate net entries / exits for the day
    MTA_turnstiles_grouped = MTA_turnstiles_sorted.groupby(["Unit"])
    MTA_turnstiles_sorted["Cumulative entries"] = pd.to_numeric(MTA_turnstiles_sorted["Cumulative entries"], errors="coerce")
    MTA_turnstiles_sorted["Cumulative exits"] = pd.to_numeric(MTA_turnstiles_sorted["Cumulative exits"], errors="coerce")
    MTA_turnstiles_sorted["Net entries"] = MTA_turnstiles_grouped["Cumulative entries"].transform(pd.Series.diff)
    MTA_turnstiles_sorted["Net exits"] = MTA_turnstiles_grouped["Cumulative exits"].transform(pd.Series.diff)

    #data cleaning (deal with edge cases like negatives, NaN, etc. )
    # (MTA_turnstiles_sorted["Net entries"].astype("float64").describe(percentiles=[.1,.25,.5,.75,.9]))
    # print(MTA_turnstiles_sorted["Net exits"].describe())
    # print('Number of negative entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'] < 0])) #caused by system resets
    # print('Number of negative exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'] < 0])) #caused by system resets
    # print('Number of NaN entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'].isnull()])) #caused by net zero difference
    # print('Number of NaN exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'].isnull()])) #caused by net zero difference

    #set outliers to zero
    MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net entries"] < 0] = 0
    MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net exits"] < 0] = 0
    MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net entries"] > 10000] = 0 #assumption: max error threshold
    MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net exits"] > 10000] = 0 #assumption: max error threshold
    MTA_turnstiles_sorted["Net entries"] = MTA_turnstiles_sorted["Net entries"].fillna(0)
    MTA_turnstiles_sorted["Net exits"] = MTA_turnstiles_sorted["Net exits"].fillna(0)

    # print('Number of negative entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'] < 0])) #caused by system resets
    # print('Number of negative exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'] < 0])) #caused by system resets
    # print('Number of NaN entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'].isnull()])) #caused by net zero difference
    # print('Number of NaN exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'].isnull()])) #caused by net zero difference

    MTA_turnstiles_sorted["Date"] = pd.to_datetime(MTA_turnstiles_sorted["Datetime"], errors="coerce", yearfirst = True).dt.strftime('%y-%m-%d')
    MTA_turnstiles_by_day = MTA_turnstiles_sorted.groupby(by=["Unit", "Date"]).agg({"Net entries": ["sum"]})
    MTA_turnstiles_by_day = MTA_turnstiles_by_day.reset_index()
    MTA_turnstiles_by_day.columns = MTA_turnstiles_by_day.columns.get_level_values(0)
    return MTA_turnstiles_sorted, MTA_turnstiles_by_day

MTA_turnstiles = raw_MTA_data_tmp.copy(deep=True)
MTA_turnstiles, MTA_turnstiles_by_day  = getNetEntriesExits(MTA_turnstiles)

In [6]:
print("Question 2: What is the total number of entries & exits across the subway system for February 1, 2013?")
print(MTA_turnstiles_by_day.loc[MTA_turnstiles_by_day["Date"] == "13-02-01"].sum()["Net entries"]) 

Question 2: What is the total number of entries & exits across the subway system for February 1, 2013?
5818588.0


In [7]:
print("Question 3: Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date?")

def getBusynessByStation(MTA_turnstiles):
    MTA_turnstiles["Busy-ness"] = MTA_turnstiles["Net entries"] + MTA_turnstiles["Net exits"] 
    MTA_stations_by_day = MTA_turnstiles.groupby(by=["Station", "Date"]).agg({"Busy-ness": ["sum"]})
    MTA_stations_by_day = MTA_stations_by_day.reset_index()
    MTA_stations_by_day.columns = MTA_stations_by_day.columns.get_level_values(0)
    return MTA_stations_by_day

MTA_stations_by_day = getBusynessByStation(MTA_turnstiles)
MTA_stations_by_dayFeb1st = MTA_stations_by_day.loc[MTA_stations_by_day["Date"] == "13-02-01"].sort_values(by="Busy-ness")
busiestStationName = MTA_stations_by_dayFeb1st.iloc[-1]["Station"]
busiestStationTransactions = int(MTA_stations_by_dayFeb1st.iloc[-1]["Busy-ness"])

print("The most busy station is {0} and it had {1} entries/exits".format(busiestStationName, busiestStationTransactions)) #likely  times square, grand central,

Question 3: Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date?
The most busy station is 34 ST-PENN STA and it had 348286 entries/exits


In [100]:
# Build this data frame for 2012 and 2013.
# rows = calendar days for that year (365)
# columns = busyness for each station, "day of week" (~300)

#1. To create each dataframe, write a method which iterates over all weeks of that year. 
#   1a. For each week, return a dataframe with rows = days, columns = busyness for each station
#   1b. append to the original dataframe
#2. groupby day to sum up total busyness (~300 x 365 rows --> ~300x 1 rows)
#3. Create a row with summation of all dates
#4. Transpose 2012 and 2013 stations x day. merge into one dataset
#5. Calculate usage growth / decline by station

def getMTAStationsByDay(dateStr):
    #turnstile x (8 hrs of data)
    raw_data = read_MTA_csv("~/Projects/NYC_MTA_DATA_ANALYSIS/data/source-csv/" + dateStr + ".csv") #reads data and adds columns
    raw_data = addStationName("../data/Remote-Booth-Station.xls", raw_data) #adds station name
    
    #turnstile x (1 hrs of data)
    raw_data = expandRows(raw_data)
    
    #turnstile x (1 day of data)
    MTA_turnstiles = raw_data.copy(deep=True)
    MTA_turnstiles, MTA_turnstiles_by_day = getNetEntriesExits(MTA_turnstiles)
    
    #station x (1 day of data)...7 days of data
    MTA_stations_by_day = getBusynessByStation(MTA_turnstiles)
    return MTA_stations_by_day





Question 4: What stations have seen the most usage growth/decline in 2013?


In [105]:
dates_2013 = pd.date_range(start='1/2/2013', freq="W-SAT", periods = 51)
station_date_busyness2013 = pd.DataFrame()
for date in dates_2013.strftime("%y%m%d"):
    station_date_busyness2013 = pd.concat([station_date_busyness2013, getMTAStationsByDay(date)])



In [107]:
dates_2012 = pd.date_range(start='1/2/2012', freq="W-SAT", periods = 51) #NOTE: manually had to change 20120714.csv since there are junk rows in the beginning 10 rows 
station_date_busyness2012 = pd.DataFrame()
for date in dates_2012.strftime("%y%m%d"):
    station_date_busyness2012 = pd.concat([station_date_busyness2012, getMTAStationsByDay(date)])
    
# station_date_busyness2012 = readMTA_CSV(station_date_busyness2013,"120714")



/opt/homebrew/Caskroom/miniforge/base/envs/twosigma/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [113]:
print("Question 4: What stations have seen the most usage growth/decline in 2013?") 
print("Top 5 stations and bottom by stations by 'busy-ness' growth / decline in 2013 (vs. 2012)") 

station_busyness2013 = station_date_busyness2013[["Station", "Busy-ness"]].groupby(by="Station").agg(["sum"])
station_busyness2012 = station_date_busyness2012[["Station", "Busy-ness"]].groupby(by="Station").agg(["sum"])

station_busyness2013.columns = station_busyness2013.columns.get_level_values(0)
station_busyness2012.columns = station_busyness2012.columns.get_level_values(0)

station_busyness = station_busyness2012.merge(station_busyness2013,on="Station",how="inner",suffixes=["_2012", "_2013"])
station_busyness["2013 delta (vs. 2012) - # change"] = station_busyness["Busy-ness_2013"] - station_busyness["Busy-ness_2012"] 
station_busyness["2013 delta (vs. 2012) - % change "] = (station_busyness["Busy-ness_2013"] - station_busyness["Busy-ness_2012"]) /  station_busyness["Busy-ness_2012"]

print(station_busyness.sort_values(by="2013 delta (vs. 2012) - # change"))
print()
#print(station_busyness.sum())

Question 4: What stations have seen the most usage growth/decline in 2013?
                 Busy-ness_2012  Busy-ness_2013  \
Station                                           
JOURNAL SQUARE       15758024.0      11657819.0   
WHITEHALL ST         15841461.0      11979171.0   
NEWARK HW BMEBE      15601192.0      13299964.0   
THIRTY ST            18561196.0      17082756.0   
CASTLE HILL AVE       3717151.0       2826447.0   
...                         ...             ...   
ROOSEVELT AVE        27652048.0      29989408.0   
FULTON ST            34859135.0      37476450.0   
ATLANTIC AVE         22794118.0      25693646.0   
34 ST-PENN STA       95204739.0      98249708.0   
42 ST-GRD CNTRL      81991428.0      85190086.0   

                 2013 delta (vs. 2012) - # change  \
Station                                             
JOURNAL SQUARE                         -4100205.0   
WHITEHALL ST                           -3862290.0   
NEWARK HW BMEBE                        -2301228.0

In [122]:
print("Question 5: What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely?")
#Goal 1: Find dates which are least busy"
#1. Using 2013 dataset from above, group by date.
print("5a) dates which are least busy:")
print("Typically the days in Q1 are the least busy, almost half as busy as the most-busy days (occuring in the holiday season around December)")

date_busyness2013 = station_date_busyness2013[["Date", "Busy-ness"]].groupby(by="Date").agg(["sum"])
date_busyness2013.columns = date_busyness2013.columns.get_level_values(0)
print(date_busyness2013.sort_values(by="Busy-ness").head(10))
#print(date_busyness2013.sort_values(by="Busy-ness").tail(10))



Question 5: What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely?
5a) dates which are least busy:
          Busy-ness
Date               
70-01-01        0.0
13-02-09  3819532.0
13-11-28  4281751.0
13-02-03  4434525.0
13-01-27  4513943.0
13-01-06  4545989.0
13-01-13  4551911.0
13-02-10  4641424.0
13-02-24  4646116.0
13-01-01  4721158.0
           Busy-ness
Date                
13-09-25  10983346.0
13-12-11  10989162.0
13-12-05  10997114.0
13-11-08  11004719.0
13-12-04  11007766.0
13-12-13  11010036.0
13-12-12  11027993.0
13-10-25  11028587.0
13-10-09  11030564.0
13-10-24  11092014.0


In [166]:
#Goal 2: Find days on which stations aren't at fully capacity or closed entirely
# 1. Use 2013 dataset from above. classify each day as a weekday or weekend.
# 2. Create two tables, representing the maximum capacity for that year. one for weekends, one for weekdays
#   2a. for each table group by station and filter by weekend/weekday (stations x max(busyness))
# 3. Then, for each station, iterate through all weekdays and weekends of 2013. return all dates where there is < 10% capacity or 0% capacity.


station_date_busyness2013["Date2"] = pd.to_datetime(station_date_busyness2013["Date"], yearfirst=True)
station_date_busyness2013["Day of week"] = station_date_busyness2013["Date2"].apply(lambda x: x.dayofweek)
station_date_busyness2013["Weekend or weekday?"] = np.where(station_date_busyness2013["Day of week"] < 5, "Weekday", "Weekend")

#get weekends
station_busyness2013_weekends = station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekend",["Station", "Busy-ness", "Weekend or weekday?"]]
station_busyness2013_weekends = station_busyness2013_weekends.groupby("Station").agg(["max"])
station_busyness2013_weekends.columns = station_busyness2013_weekends.columns.get_level_values(0)
#print(station_busyness2013_weekends)

#get weekdays
station_busyness2013_weekdays = station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekday",["Station", "Busy-ness", "Weekend or weekday?"]]
station_busyness2013_weekdays = station_busyness2013_weekdays.groupby("Station").agg(["max"])
station_busyness2013_weekdays.columns = station_busyness2013_weekdays.columns.get_level_values(0)
#print(station_busyness2013_weekdays)


station_busyness2013_weekends = station_busyness2013_weekends.to_dict()["Busy-ness"]
station_busyness2013_weekdays = station_busyness2013_weekdays.to_dict()["Busy-ness"]


5b) dates on which stations aren't at full capacity or are closed entirely:


In [193]:
print("5b) dates on which stations aren't at full capacity or are closed entirely:")

station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekend","Max capacity for station"] = station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekend","Station"].map(station_busyness2013_weekends)
station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekday","Max capacity for station"] = station_date_busyness2013.loc[station_date_busyness2013["Weekend or weekday?"] == "Weekday","Station"].map(station_busyness2013_weekdays)
station_date_busyness2013["Busy-ness"] = station_date_busyness2013["Busy-ness"].astype("float")
station_date_busyness2013["Max capacity for station"] = station_date_busyness2013["Max capacity for station"].astype("float")
station_date_busyness2013["% capacity used"] = (station_date_busyness2013["Busy-ness"] / station_date_busyness2013["Max capacity for station"])

print("In 2013, there were {0} closures of stations, where there was zero traffic registered".format(station_date_busyness2013.loc[station_date_busyness2013["% capacity used"] == 0.0].shape[0]))
print(station_date_busyness2013.loc[station_date_busyness2013["% capacity used"] == 0.0,["Station", "Date", "% capacity used"]])
print("------------------------------------------------------------")
print("In 2013, there were {0} occurences where stations had lower capacity (between 0% and 75% capacity)".format(station_date_busyness2013.loc[(station_date_busyness2013["% capacity used"] > 0.0) & (station_date_busyness2013["% capacity used"] < .30)].shape[0]))
print(station_date_busyness2013.loc[(station_date_busyness2013["% capacity used"] > 0.0) & (station_date_busyness2013["% capacity used"] < .30),["Station", "Date", "% capacity used"]])

In 2013, there were 246 closures of stations, where there was zero traffic registered
              Station      Date  % capacity used
967       BEACH 90 ST  12-12-29              0.0
968       BEACH 90 ST  12-12-30              0.0
969       BEACH 90 ST  12-12-31              0.0
971       BEACH 90 ST  13-01-02              0.0
973       BEACH 90 ST  13-01-04              0.0
...               ...       ...              ...
1555  FRESH POND ROAD  13-12-07              0.0
1856    KNICKERBOCKER  13-12-07              0.0
2303       SENECA AVE  13-12-08              0.0
2444  VAN ALSTON-21ST  13-12-14              0.0
2445  VAN ALSTON-21ST  13-12-15              0.0

[246 rows x 3 columns]
------------------------------------------------------------
In 2013, there were 4466 occurences where stations had lower capacity (between 0% and 75% capacity)
              Station      Date  % capacity used
66    116 ST-COLUMBIA  12-12-31         0.245681
67    116 ST-COLUMBIA  13-01-01         0.2